# Subfields

In this tutorial, we demonstrate several operations that can be used for extracting parts of the field. More precisely, we are going to show how to:

1. Get an individual component of a vector field
2. Extract smaller region of the field
    1. without resampling
    2. with resampling
    
Let us first create a vector field to work with:

In [1]:
import discretisedfield as df

p1 = (0, 0, 0)
p2 = (100, 100, 100)
cell = (2, 2, 2)
region = df.Region(p1=p1, p2=p2)
mesh = df.Mesh(region=region, cell=cell)
field = df.Field(mesh, dim=3, value=(-1, 2, 6))

## Getting a single component

From a vector field we can get its $x$, $y$, or $z$ component by simply asking for the right property. For instance:

In [2]:
field.x

Field(mesh=Mesh(region=Region(p1=(0, 0, 0), p2=(100, 100, 100)), n=(50, 50, 50), bc='', subregions={}), dim=1)

In [3]:
field.y

Field(mesh=Mesh(region=Region(p1=(0, 0, 0), p2=(100, 100, 100)), n=(50, 50, 50), bc='', subregions={}), dim=1)

In [4]:
field.z

Field(mesh=Mesh(region=Region(p1=(0, 0, 0), p2=(100, 100, 100)), n=(50, 50, 50), bc='', subregions={}), dim=1)

They all result in a scalar field and from this point on, we can perform all usual operations on fields. For example,

In [5]:
df.integral(field.x * df.dV)

-1000000.0

## Extracting smaller region

Let us say we are not interested in the entire field but only in its smaller portion - only some discretisation cells. In that case, we have two options. Before we discuss them, let us first define what we mean by "aligned" meshes:

- Mesh A is aligned to mesh B if and only if all cell coordinates of mesh A are also the coordinates of (some) cells in mesh B.

There is `|` operator which checks that. Let us have a look at a few meshes:

In [6]:
mesh1 = df.Mesh(region=df.Region(p1=(0, 0, 0), p2=(10, 10, 10)), cell=(1, 1, 1))
mesh2 = df.Mesh(region=df.Region(p1=(3, 3, 3), p2=(6, 6, 6)), cell=(1, 1, 1))
mesh3 = df.Mesh(region=df.Region(p1=(0, 0, 0), p2=(10, 10, 10)), cell=(2, 2, 2))
mesh4 = df.Mesh(region=df.Region(p1=(3.5, 3.5, 3.5), p2=(6.5, 6.5, 6.5)), cell=(1, 1, 1))

Let us now have a look if those meshes are aligned:

In [7]:
mesh1 | mesh2

True

In [8]:
mesh1 | mesh3  # discretisation cell is different

False

In [9]:
mesh1 | mesh4  # although discretisation cell is the same, mesh4 is shifted in space by 0.5

False

### Extracting subfield on aligned mesh

If we want to get a subfield whose mesh is aligned to the field we want to take part of, we use `[]` operator. The resulting field is going to have a minimum-sized mesh which contains the region we pass as an argument.

In [10]:
subregion = df.Region(p1=(1.5, 2.2, 3.9), p2=(6.1, 5.9, 9.9))
field[subregion]

Field(mesh=Mesh(region=Region(p1=(0.0, 2.0, 2.0), p2=(8.0, 6.0, 10.0)), n=(4, 2, 4), bc='', subregions={}), dim=3)

We can see that the resulting field's mesh has the minimum dimesions aligned mesh should have in order to contain the `subregion`. The resulting field has the same discretisation cell as the original one.

### Extracting field on any mesh

If we want to extact part of the field on any mesh which is contained inside the field, we do that by "resampling". We create a new field on a submesh and pass the field we want take subfield from as `value`.

In [11]:
subregion = df.Region(p1=(1.5, 2.5, 3.5), p2=(5.5, 5.5, 6.5))
submesh = df.Mesh(region=subregion, cell=(0.5, 0.5, 0.5))
df.Field(submesh, dim=3, value=field)

Field(mesh=Mesh(region=Region(p1=(1.5, 2.5, 3.5), p2=(5.5, 5.5, 6.5)), n=(8, 6, 6), bc='', subregions={}), dim=3)

One could ask why don't we always use resampling because it is a generalised case. The reason is because computing a subfield using `[]` operator is much faster.